In [32]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import (DataLoader,)
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [34]:
#hyperparameters
input_size = 28 #dimension of image
hidden_size =256 # num neurons
num_layers =2 
num_classes = 10 
sequence_length =28 #dimension of one input 
learning_rate = 0.005
batch_size = 64
num_epochs =3 

In [48]:
#fc = RNN fully connected #many to one
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first =True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)
        
    def forward(self,x):
        #set initial hidden and cell states
        h0= torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        #forward propagate LSTM
        out,_ = self.rnn(x,h0) # model with input size, with 1st hidden layer
        out = out.reshape(out.shape[0],-1) #make it in one series shape
        
        #decode the hidden state for the last time step
        out = self.fc(out)
        return out
    

    
#fc = RNN_GRU fully connected #many to one
class RNN_GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_GRU, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first =True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)
        
    def forward(self,x):
        #set initial hidden and cell states
        h0= torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        #forward propagate LSTM
        out,_ = self.gru(x,h0) # model with input size, with 1st hidden layer
        out = out.reshape(out.shape[0],-1) #make it in one series shape
        
        #decode the hidden state for the last time step
        out = self.fc(out)
        return out
        

#fc = RNN_LSTM fully connected #many to one
class RNN_LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN_LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first =True)
        self.fc = nn.Linear(hidden_size * sequence_length, num_classes)
        
    def forward(self,x):
        #set initial hidden and cell states
        h0= torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0= torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        #forward propagate LSTM
        out,_ = self.lstm(
            x,(h0,c0)
        ) # model with input size, with 1st hidden layer
        out = out.reshape(out.shape[0],-1) #make it in one series shape
        
        #decode the hidden state for the last time step
        out = self.fc(out)
        return out


In [49]:
train_dataset = datasets.MNIST (
root ="dataset/", train=True, transform=transforms.ToTensor(), download=True
)

test_dataset = datasets.MNIST(
root ="dataset/", train = False, transform=transforms.ToTensor(), download=True
)

train_loader =DataLoader(dataset =train_dataset, batch_size = batch_size, \
                        shuffle=True)
test_loader = DataLoader(dataset= test_dataset, batch_size = batch_size,\
                        shuffle =True
                        )

In [50]:
#initialize model
model = RNN_LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

In [51]:
#loss and optimizer for gradient descent
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [52]:
#train network
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
        data = data.to(device=device).squeeze(1)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent update step/adam step
        optimizer.step()

100%|██████████| 938/938 [04:58<00:00,  3.14it/s]


In [53]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Accuracy on training set: 98.876663
Accuracy on test set: 98.62
